<a href="https://colab.research.google.com/github/emilycalvert/stipend_iquitos/blob/main/Iquitos2023housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loreto- Maynas: Housing Cost**

_Variable definitions, methodologies, and original survey forms can be accessed in the INEI repository as "Diccionarios", "Ficha-Tecnica", and "CED-01-100-2023" correspondingly._

## [Data can be accessed at here.](https://proyectos.inei.gob.pe/microdatos/Consulta_por_Encuesta.asp)

## **Processing**

## Methods:
1. Upload data for Trimester 1, 2, and 3.
2. Join the three dataset on common keys after comparing columns.
3. Inspect elements of the resulting dataframe.
4. Print zipcodes represented in the housing data. Filter for Iquitos and surrounding areas.
5. Inspect the sample size and characteristics of data related to Iquitos.
6. Ensure entries fall within a relevant timeframe.
7. Solve none values.
8. Coerce data to numerical for machine processing.
9. Cut columns of interest to relevancy.
10. Convert columns to appropriate datatypes for the corresponding indicators.

## Joins, Filters, Drops

In [ ]:
#Upload data
import pandas as pd
import io
from google.colab import files
import warnings
warnings.filterwarnings("ignore")

In [ ]:
tri1 = files.upload()

In [ ]:
tri2 = files.upload()

In [ ]:
tri3 = files.upload()

In [ ]:
tri1name = list(tri1.keys())[0]
tri2name = list(tri2.keys())[0]
tri3name = list(tri3.keys())[0]

#Try reading with Latin-1 encoding
tri1df = pd.read_csv(io.BytesIO(tri1[tri1name]), encoding='ISO-8859-1')
tri2df = pd.read_csv(io.BytesIO(tri2[tri2name]), encoding='ISO-8859-1')
tri3df = pd.read_csv(io.BytesIO(tri3[tri3name]), encoding='ISO-8859-1')

In [ ]:
#Set option to display all columns
pd.set_option('display.max_columns', None)

#Print all column names
print(tri1df.columns.tolist())

have_same_columns = (set(tri1df.columns) == set(tri2df.columns)) and (set(tri2df.columns) == set(tri3df.columns))
#Print the result
print(have_same_columns)

In [ ]:
#Compare column order directly
same_order = (list(tri1df.columns) == list(tri2df.columns)) and (list(tri2df.columns) == list(tri3df.columns))

#If the order is not the same, print the differing columns
if not same_order:
    print("Columns in tri1df:", tri1df.columns)
    print("Columns in tri2df:", tri2df.columns)
    print("Columns in tri3df:", tri3df.columns)
else:
    print("All DataFrames have the same columns in the same order.")

In [ ]:
extra_columns1a = []
extra_columns1b = []
extra_columns2a = []
extra_columns2b = []
extra_columns3a = []
extra_columns3b = []

for column in tri1df.columns:
    if column not in tri2df.columns:
        extra_columns1a.append(column)

for column in tri1df.columns:
    if column not in tri3df.columns:
        extra_columns1b.append(column)


for column in tri2df.columns:
    if column not in tri1df.columns:
        extra_columns2a.append(column)

for column in tri2df.columns:
    if column not in tri3df.columns:
        extra_columns2b.append(column)

for column in tri3df.columns:
    if column not in tri1df.columns:
        extra_columns3a.append(column)

for column in tri3df.columns:
    if column not in tri2df.columns:
        extra_columns3b.append(column)

# Print the extra column(s)
print("column(s) in tri1df not in tri2df:", extra_columns1a)
print("column(s) in tri1df not in tri2df:", extra_columns1b)
print("column(s) in tri2df not in tri1df:", extra_columns2a)
print("column(s) in tri2df not in tri3df:", extra_columns2b)
print("column(s) in tri3df not in tri1df:", extra_columns3a)
print("column(s) in tri3df not in tri2df:", extra_columns3b)


Remove inconsistent columns to allow a join that retains data integrity and readability.
P114A details is the home has digital television. Considering its irrelevance it was resolved by dropping the column.

In [ ]:
tri2df = tri2df.drop(columns=['P114A'])
combined_df = pd.concat([tri1df, tri2df, tri3df], ignore_index=True)

Descriptive statistics of the combined data.

In [ ]:
# @title
combined_df['UBIGEO'] = combined_df['UBIGEO'].astype(int)

range1 = range(160101, 160114)
range2 = range(160301, 160306)
range3 = range(160801, 160805)

all_ranges = list(range1) + list(range2) + list(range3)

#Filter the DataFrame for these ranges
filtered_df = combined_df[combined_df['UBIGEO'].isin(all_ranges)]

filtered_df.describe()

Descriptive statistics and sample sizes in Iquitos and surrounding UBIGEO codes.

In [ ]:
# @title
print(filtered_df['UBIGEO'].unique())
print(filtered_df['UBIGEO'].describe())

[160108 160103 160101 160112 160113 160301 160303 160106 160105 160107
 160305 160104 160110 160102 160304]
count       827.000000
mean     160128.453446
std          60.869238
min      160101.000000
25%      160103.000000
50%      160108.000000
75%      160113.000000
max      160305.000000
Name: UBIGEO, dtype: float64


In [ ]:
# @title
ubigeo_coordinates = {
    "160101": (-3.74805555555556, -73.2441666666667),
    "160108": (-3.72861111111111, -73.2419444444445	),
    "160103": (	-4.00166666666667, -73.1569444444444),
    "160112": (-3.76916666666667, 	-73.26),
    "160113": (-3.77027777777778, -73.2802777777778),
    "160301": (	-4.50138888888889, -73.5694444444444),
    "160303": (-3.48972222222222, -74.7816666666667),
    "160106": (-3.48861111111111, -73.0816666666667),
    "160105": (-3.42305555555556, -72.764444444444507),
    "160107": (-2.48916666666667, -73.6761111111111),
    "160305": (-4.5875, -74.7672222222222),
    "160104": (-3.50027777777778, -73.0411111111111),
    "160110": (-0.970555555555556	, -75.1741666666667),
    "160102": (-3.88833333333333, -73.6975),
    "160304": (-3.805, -75.0605555555556)}


Ensure lat and long for UBIGEO codes are properly appended to the dataframe.

In [ ]:
# @title
#Convert UBIGEO codes to strings if they are not already
filtered_df['UBIGEO'] = filtered_df['UBIGEO'].astype(str)

#Function to get latitude and longitude
def get_lat_lon(ubigeo):
    return ubigeo_coordinates.get(ubigeo, (None, None))

#Append latitude and longitude to the DataFrame
filtered_df['Latitude'] = filtered_df['UBIGEO'].apply(lambda x: get_lat_lon(x)[0])
filtered_df['Longitude'] = filtered_df['UBIGEO'].apply(lambda x: get_lat_lon(x)[1])

#Display the updated DataFrame
filtered_df.head()

,AÑO,MES,CONGLOME,VIVIENDA,HOGAR,UBIGEO,DOMINIO,ESTRATO,PERIODO,TIPENC,FECENT,RESULT,PANEL,P22,P23,P24A,P24B,P25$1,P25$2,P25$3,P25$4,P25$5,P101,P102,P103,P103A,P104,P104A,P104B1,P104B2,P105A,P105B,P106,P106A,P106B,P107B1,P107C11,P107C12,P107C13,P107C14,P107C16,P107C17,P107C18,P107C19,P107C110,P107D1,P107B2,P107C21,P107C22,P107C23,P107C24,P107C26,P107C27,P107C28,P107C29,P107C210,P107D2,P107B3,P107C31,P107C32,P107C33,P107C34,P107C36,P107C37,P107C38,P107C39,P107C310,P107D3,P107B4,P107C41,P107C42,P107C43,P107C44,P107C46,P107C47,P107C48,P107C49,P107C410,P107D4,P107E,P110,P110A1,P110A,P110A_MODIFICADA,P110C,P110C1,P110C2,P110C3,P110D,P110E,P110F,P110G,P111A,P1121,P1123,P1124,P1125,P1126,P1127,P112A,P1131,P1132,P1133,P1135,P1136,P1139,P1137,P1138,P113A,P1141,P1142,P1143,P1144,P1145,P114A,P114B1,P114B2,P114B3,P1171$01,P1171$02,P1171$04,P1171$05,P1171$06,P1171$07,P1171$08,P1171$09,P1171$10,P1171$11,P1171$12,P1171$13,P1171$14,P1171$15,P1171$16,P1171$17,P1172$01,P1172$02,P1172$04,P1172$05,P1172$06,P1172$07,P1172$08,P1172$09,P1172$10,P1172$11,P1172$12,P1172$13,P1172$14,P1172$15,P1172$16,P1172$17,P1173$01,P1173$02,P1173$04,P1173$05,P1173$06,P1173$07,P1173$08,P1173$09,P1173$10,P1173$11,P1173$12,P1173$13,P1173$14,P1173$15,P1173$16,P1173$17,P1174$01,P1174$02,P1174$04,P1174$05,P1174$06,P1174$07,P1174$08,P1174$09,P1174$10,P1174$11,P1174$12,P1174$13,P1174$14,P1174$15,P1174$16,P1174$17,P1175$01,P1175$02,P1175$04,P1175$05,P1175$06,P1175$07,P1175$08,P1175$09,P1175$10,P1175$11,P1175$12,P1175$13,P1175$14,P1175$15,P1175$16,P1175$17,P117T2,P117T3,P117T4,T110,P200I,P600I,P600D1,P600M1,P600A1,P600D2,P600M2,P600A2,P612I1,P612I11,P612I2,P612I22,P700I,P710I,P800I,P110I,TICUEST01,D105B,D106,D107D1,D107D2,D107D3,D107D4,D1172$01,D1173$01,D1174$01,D1172$02,D1173$02,D1174$02,D1172$04,D1173$04,D1174$04,D1172$05,D1173$05,D1174$05,D1172$06,D1173$06,D1174$06,D1172$07,D1173$07,D1174$07,D1172$08,D1173$08,D1174$08,D1172$09,D1173$09,D1174$09,D1172$10,D1173$10,D1174$10,D1172$15,D1173$15,D1174$15,D1172$16,D1173$16,D1174$16,D612I11,D1172$11,D1173$11,D1174$11,D1172$12,D1173$12,D1174$12,D1172$13,D1173$13,D1174$13,D1172$14,D1173$14,D1174$14,D1172$17,D1173$17,D1174$17,D612I22,I105B,I106,I1172$01,I1172$02,I1172$04,I1172$05,I1172$06,I1172$07,I1172$08,I1172$09,I1172$10,I1172$11,I1172$12,I1172$13,I1172$14,I1172$17,I1172$15,I1172$16,I1173$01,I1174$01,I1173$02,I1174$02,I1173$04,I1174$04,I1173$05,I1174$05,I1173$06,I1174$06,I1173$07,I1174$07,I1173$08,I1174$08,I1173$09,I1174$09,I1173$10,I1174$10,I1173$11,I1174$11,I1173$12,I1174$12,I1173$13,I1174$13,I1173$14,I1174$14,I1173$15,I1174$15,I1173$16,I1174$16,I1173$17,I1174$17,T111A,NBI1,NBI2,NBI3,NBI4,NBI5,FACTOR,Latitude,Longitude
326,2023,1,10269,8,11,160108,7,2,2,3,20230119,1,1,2,.,1,1,1,0,1,1,0,1,1,5,4,5,4,3,3,2,,600,1,1,2,,,,,,,,,,,2,,,,,,,,,,,2,,,,,,,,,,,2,,,,,,,,,,,,1,1,2,.400000005960464,1,3,,,1,1,1,1,1,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,2,0,1,1,1,0,,0,1,1,1,1,1,0,0,1,0,0,0,0,1,1,1,0,0,0,53,90,38,,,4,,,,,105,51,0,,,,0,0,0,,,0,,,,,0,0,0,,,,,,0,,,0,,,,,,,,,,,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,341,0,0,1,02,02,03,01,2023,17,01,2023,2,,2,,02,02,02,02,2,,359,,,,,32,0,,54,0,,23,0,0,,,,,,,2,0,0,,,,,,,,,,,,,,,,,,,,58,0,,28,0,,0,0,,,,,,,359,32,54,23,,,2,,,,,58,28,0,,,,0,,0,,0,0,,,,,0,0,,,,,,,,,0,,0,,0,,,,,,,,1,0,0,0,0,0,634.889059,-3.728611,-73.241944
327,2023,1,10269,21,11,160108,7,2,2,3,20230127,1,1,1,160401,1,1,1,0,1,1,0,1,1,5,4,4,3,2,2,2,,800,1,1,2,,,,,,,,,,,2,,,,,,,,,,,2,,,,,,,,,,,2,,,,,,,,,,,,1,1,1,1.20000004768372,1,2,,,2,1,1,1,1,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,2,0,1,1,1,0,,0,1,0,1,1,1,0,0,1,0,0,0,0,1,1,1,0,0,0,52,72,34,,,5,,,,,29,52,0,,,,0,0,0,,,0,,,,,0,0,0,,,,,,0,,,0,,,,,,,,,,,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,244,0,0,1,01,01,05,01,2023,19,01,2023,2,,2,,01,01,01,01,2,,479,,,,,31,0,,43,0,,20,0,0,,,,,,,3,0,0,,,,,,,,,,,,,,,,,,,,16,0,,29,0,,0,0,,,,,,,479,31,43,20,,,3,,,,,16,29,0,,,,0,,0,,0,0,,,,,0,0,,,,,,,,,0,,0,,0,,,,,,,,1,0,0,0,0,0,634.889059,-3.728611,-73.241944
328,2023,1,10269,32,11,160108,7,2,2,3,20230122,1,1,2,.,1,1,1,0,1,1,0,1,1,3,4,5,3,1,1,2,

Visualize sample distribution: Hover over map marker to reveal the sample size within that UBIGEO code.

## **Visualize the Geographical Sample Distribution**

In [ ]:
# @title
import folium

new_map = folium.Map(location=[-3.75, -73.25], zoom_start=10)

# Calculate the count for each UBIGEO code
ubigeo_counts = filtered_df['UBIGEO'].value_counts()

# Iterate through the DataFrame and add markers
for index, row in filtered_df.drop_duplicates('UBIGEO').iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    ubigeo_code = row['UBIGEO']
    count = ubigeo_counts[ubigeo_code]
    if lat is not None and lon is not None:
        # Create a popup text with the count for the UBIGEO code
        popup_text = f"Count: {count}"
        # Add marker with popup to the new map
        folium.Marker([lat, lon], popup=popup_text).add_to(new_map)

# Display the new map
new_map

1. Identify none values.
2. Considering the irrelevancy or redundancy of these columns they were dropped:
  * P1175 01: 754- El último gasto mensual por consumo de: Agua
  * P1175 02: 816- El último gasto mensual por consumo de: Electricidad
  * P1175 13: 809- El último gasto mensual por consumo de: TV Cable o satelital
  * P1175 14: 363- El último gasto mensual por consumo de: Internet
  * P1175 15: 700- El último gasto mensual por consumo de: Otro

In [ ]:
# @title
pd.set_option('display.max_rows', None)

zero_counts = {}

for column in filtered_df.columns:
  if filtered_df[column].nunique() > 2:
    zero_counts[column] = (filtered_df[column] == 0).sum()

for column, count in zero_counts.items():
    if count != 0:
        print(f"{column}: {count}")

In [ ]:
# @title
#List of columns to be removed
columns_to_remove = ['P1175$01', 'P1175$02', 'P1175$13', 'P1175$14', 'P1175$15']

#Remove the specified columns from the DataFrame
filtered_df.drop(columns=columns_to_remove, inplace=True)

The following logic was applied for data type configuration:
1. Any column with 2 unique values present was a True/False (Bool).
2. If the column had more than 2 unique values it was converted to a catergorical value and coerced to an integer afterwards.

In [ ]:
# @title
for column in filtered_df.columns:
    unique_values = filtered_df[column].unique()
    num_unique = len(unique_values)

    #If the column has only 0 and 1
    if num_unique == 2 and all(val in [0, 1] for val in unique_values):
        combined_df[column] = filtered_df[column].astype(bool)

    #If the column has values 0, 1, 3
    elif num_unique == 3 and all(val in [0, 1, 3] for val in unique_values):
        filtered_df[column] = pd.Categorical(filtered_df[column], categories=[0, 1, 3])

    #If the column has more than 3 unique values
    elif num_unique > 3:
        filtered_df[column] = filtered_df[column].astype('category')

    #Else, ensure it is of integer type
    else:
        filtered_df[column] = pd.to_numeric(filtered_df[column], downcast='integer', errors='coerce')

In [ ]:
# @title
columns_of_interest = [ 'TIPENC', 'FECENT', 'P24A', 'P24B', 'P25$1', 'P25$2', 'P25$3', 'P25$4', 'P25$5', 'P101', 'P102',
    'P103', 'P103A', 'P104', 'P104A', 'P104B1', 'P104B2', 'P105A', 'P105B', 'P106',
    'P106A', 'P106B', 'P107B1', 'P110', 'P110A1', 'P110A', 'P110C', 'P110C1', 'P110F',
    'P111A', 'P1121', 'P112A', 'P1142', 'P1171$01', 'P1171$02', 'P1171$04', 'P1171$11',
    'P1171$12', 'P1171$14', 'P1171$15', 'P1172$01', 'P1172$02', 'P1172$04', 'P1172$12',
    'P1172$14', 'P1172$15', 'P1172$17', 'P1173$01', 'P1173$02', 'P1173$04', 'P1173$12',
    'P1173$14', 'P1173$15', 'P1175$04', 'P117T2', 'P117T3',
    'P117T4', 'Latitude', 'Longitude']
filtered_df = filtered_df[columns_of_interest]

Common conventions for column description.

In [ ]:
# @title
column_descriptions = {
    "FECENT": "Fecha de Resultado final de la encuesta ( Mes/Día/Año)",
    "TIPENC": "Tipo de selección del conglomerado",
    "P24A": "Exterior Walls: Plastered",
    "P24B": "Exterior Walls: Painted",
    "P25$1": "Street Type: Dirt Road",
    "P25$2": "Street Type: Asphalted Road",
    "P25$3": "Street Type: Sidewalks",
    "P25$4": "Street Type: Public Lighting",
    "P25$5": "Street Type: None",
    "P101": "Housing Type",
    "P102": "Main Material of Exterior Walls",
    "P103": "Main Material of Floors",
    "P103A": "Main Material of Roofs",
    "P104": "Total Rooms Excluding Bathroom, Kitchen, Passages, Garage",
    "P104A": "Rooms Used Exclusively for Sleeping",
    "P104B1": "Construction License",
    "P104B2": "Built with Technical Assistance",
    "P105A": "Home Ownership Status",
    "P105B": "Monthly Rent or Purchase Amount",
    "P106": "Estimated Monthly Rental Value",
    "P106A": "Home Has Property Title",
    "P106B": "Property Title Registered in SUNARP",
    "P107B1": "Credit or Loan for Home Purchase",
    "P110": "Main Source of Water",
    "P110A1": "Is Water Drinkable",
    "P110A": "Chlorine Residual Level",
    "P110C": "Daily Access to Water Service",
    "P110C1": "Hours of Water Service Per Day",
    "P110F": "Payment for Water Service",
    "P111A": "Bathroom Connection Type",
    "P1121": "Home Lighting Type: Electricity",
    "P112A": "Electric Service Type",
    "P1142": "Home Has Cell Phone",
    "P1171$01": "Last Monthly Water Consumption Expense",
    "P1171$02": "Last Monthly Electricity Consumption Expense",
    "P1171$04": "Last Monthly Gas (LPG) Consumption Expense",
    "P1171$11": "Last Monthly Telephone Expense",
    "P1171$12": "Last Monthly Mobile Phone Expense",
    "P1171$14": "Last Monthly Internet Expense",
    "P1171$15": "Last Monthly Expense: Other",
    "P1172$01": "Monthly Water Expense Paid by Household Member",
    "P1172$02": "Monthly Electricity Expense Paid by Household Member",
    "P1172$04": "Monthly Gas (LPG) Expense Paid by Household Member",
    "P1172$12": "Monthly Mobile Phone Expense Paid by Household Member",
    "P1172$14": "Monthly Internet Expense Paid by Household Member",
    "P1172$15": "Monthly Other Expense Paid by Household Member",
    "P1172$17": "Monthly Portable Internet Expense Paid by Household Member",
    "P1173$01": "Monthly Water Expense Gifted or Donated",
    "P1173$02": "Monthly Electricity Expense Gifted or Donated",
    "P1173$04": "Monthly Gas (LPG) Expense Gifted or Donated",
    "P1173$12": "Monthly Mobile Phone Expense Gifted or Donated",
    "P1173$14": "Monthly Internet Expense Gifted or Donated",
    "P1173$15": "Monthly Other Expense Gifted or Donated",
    "P1175$04": "Last Monthly Gas (LPG) Consumption Expense",
    "P117T2": "Total Monthly Expenses Paid by Household",
    "P117T3": "Total Monthly Expenses Donated to Household",
    "P117T4": "Total Monthly Expenses from Self-Supply",
    'Latitude': 'Latitude',
    'Longitude':'Longitude'
    }

In [ ]:
# @title
filtered_df = filtered_df.rename(columns=column_descriptions)

Inspect final filtered dataframe.


In [ ]:
# @title
filtered_df.head()

,Tipo de selección del conglomerado,Fecha de Resultado final de la encuesta ( Mes/Día/Año),Exterior Walls: Plastered,Exterior Walls: Painted,Street Type: Dirt Road,Street Type: Asphalted Road,Street Type: Sidewalks,Street Type: Public Lighting,Street Type: None,Housing Type,Main Material of Exterior Walls,Main Material of Floors,Main Material of Roofs,"Total Rooms Excluding Bathroom, Kitchen, Passages, Garage",Rooms Used Exclusively for Sleeping,Construction License,Built with Technical Assistance,Home Ownership Status,Monthly Rent or Purchase Amount,Estimated Monthly Rental Value,Home Has Property Title,Property Title Registered in SUNARP,Credit or Loan for Home Purchase,Main Source of Water,Is Water Drinkable,Chlorine Residual Level,Daily Access to Water Service,Hours of Water Service Per Day,Payment for Water Service,Bathroom Connection Type,Home Lighting Type: Electricity,Electric Service Type,Home Has Cell Phone,Last Monthly Water Consumption Expense,Last Monthly Electricity Consumption Expense,Last Monthly Gas (LPG) Consumption Expense,Last Monthly Telephone Expense,Last Monthly Mobile Phone Expense,Last Monthly Internet Expense,Last Monthly Expense: Other,Monthly Water Expense Paid by Household Member,Monthly Electricity Expense Paid by Household Member,Monthly Gas (LPG) Expense Paid by Household Member,Monthly Mobile Phone Expense Paid by Household Member,Monthly Internet Expense Paid by Household Member,Monthly Other Expense Paid by Household Member,Monthly Portable Internet Expense Paid by Household Member,Monthly Water Expense Gifted or Donated,Monthly Electricity Expense Gifted or Donated,Monthly Gas (LPG) Expense Gifted or Donated,Monthly Mobile Phone Expense Gifted or Donated,Monthly Internet Expense Gifted or Donated,Monthly Other Expense Gifted or Donated,Last Monthly Gas (LPG) Consumption Expense,Total Monthly Expenses Paid by Household,Total Monthly Expenses Donated to Household,Total Monthly Expenses from Self-Supply,Latitude,Longitude
326,3,20230119,1,1,1.0,0.0,1.0,1.0,0.0,1,1,5,4,5,4,3,3,2,,600,1,1.0,2.0,1,1.0,2,1.0,3,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,53,90,38,105,0,,,0,0,0,0,0.0,,0,341,0,0,-3.728611,-73.241944
327,3,20230127,1,1,1.0,0.0,1.0,1.0,0.0,1,1,5,4,4,3,2,2,2,,800,1,1.0,2.0,1,1.0,1,1.0,2,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,52,72,34,29,0,,,0,0,0,0,0.0,,0,244,0,0,-3.728611,-73.241944
328,3,20230122,1,1,1.0,0.0,1.0,1.0,0.0,1,1,3,4,5,3,1,1,2,,800,1,1.0,2.0,1,1.0,3,1.0,2,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,58,165,70,41,0,,,0,0,0,50,0.0,,0,338,50,0,-3.728611,-73.241944
329,3,20230117,1,3,1.0,0.0,1.0,1.0,0.0,1,9,5,4,4,3,1,2,2,,300,2,NaN,2.0,1,1.0,3,1.0,24,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,68,180,32,46,0,,,0,0,0,0,0.0,,0,378,0,0,-3.728611,-73.241944
330,3,20230119,1,1,1.0,0.0,1.0,1.0,0.0,1,1,3,4,2,1,1,1,2,,400,1,1.0,2.0,1,1.0,3,1.0,4,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0,0,68,0,0,,,57,120,0,29,0.0,,0,68,278,0,-3.728611,-73.241944


In [ ]:
# @title
list(filtered_df.columns)

In housing type:

*   1= Casa independiente
*   2= Departamento en edificio
*   3= Vivienda en quinta
*   4= Vivienda en casa de vecindad(callejón solar o corralón)
*   5= Choza o cabaña
*   6= Vivienda improvisada
*   7= Local no destinado para habitación humana

It is reasonable based on the statistics to assume that that missing value is 2 to represent apartments to that was mapped across the dataframe.

In [ ]:
# @title
#Basic Statistics for 'Estimated Monthly Rental Value'
print("Basic Statistics for 'Estimated Monthly Rental Value':")
print(filtered_df['Estimated Monthly Rental Value'].describe())

#Basic Statistics for 'Housing Type'
print("\nBasic Statistics for 'Housing Type':")
print(filtered_df['Housing Type'].value_counts())

Basic Statistics for 'Estimated Monthly Rental Value':
count     827
unique     32
top          
freq      247
Name: Estimated Monthly Rental Value, dtype: object

Basic Statistics for 'Housing Type':
1    551
     239
5     32
4      3
3      2
Name: Housing Type, dtype: int64


In [ ]:
# @title
#Replace NaN or None values with '2' in 'Housing Type' column
filtered_df['Housing Type'] = filtered_df['Housing Type'].fillna(2)


#In case 'Housing Type' has string representations of missing values
filtered_df['Housing Type'] = filtered_df['Housing Type'].replace({'': 2, 'NaN': 2, 'None': 2})

#Convert to the appropriate type if necessary
filtered_df['Housing Type'] = pd.to_numeric(filtered_df['Housing Type'], errors='coerce').fillna(2).astype(int)
#Convert to the appropriate type if necessary
filtered_df['Estimated Monthly Rental Value'] = pd.to_numeric(filtered_df['Estimated Monthly Rental Value'], errors='coerce')
filtered_df['Total Monthly Expenses Donated to Household'] = pd.to_numeric(filtered_df['Total Monthly Expenses Donated to Household'], errors='coerce')
filtered_df['Total Monthly Expenses Paid by Household'] = pd.to_numeric(filtered_df['Total Monthly Expenses Paid by Household'], errors='coerce')

#Verify the replacement
print("\nUpdated Basic Statistics for 'Housing Type':")
print(filtered_df['Housing Type'].value_counts())
regression = filtered_df.copy

In [ ]:
# @title
print(filtered_df.columns.tolist())

In [ ]:
# @title
!pip install plotly

I added the column for self paid expenses to the columne for donated expenses to respresent total monthly expenses. Now we have a "Total Expenses" column.

In [ ]:
# @title
filtered_df = filtered_df
filtered_df['Total Expenses'] = filtered_df['Total Monthly Expenses Paid by Household'] + filtered_df['Total Monthly Expenses Donated to Household']
filtered_df['Total Expenses'].describe()

count     613.000000
mean      218.745514
std       175.921924
min         0.000000
25%        87.000000
50%       186.000000
75%       313.000000
max      1136.000000
Name: Total Expenses, dtype: float64

Here a description of the rental value column is displayed.

In [ ]:
# @title
filtered_df['Estimated Monthly Rental Value'].describe()

count      580.000000
mean      2464.660345
std      14193.895607
min         10.000000
25%        150.000000
50%        300.000000
75%        600.000000
max      99999.000000
Name: Estimated Monthly Rental Value, dtype: float64

In [ ]:
# @title
#Replace NaN values with 0
filtered_df['Estimated Monthly Rental Value'].fillna(0, inplace=True)

#Replace values of 99999 with 0
filtered_df['Estimated Monthly Rental Value'] = filtered_df['Estimated Monthly Rental Value'].replace(99999, 0)

#Truncate decimals (convert to integers)
filtered_df['Estimated Monthly Rental Value'] = filtered_df['Estimated Monthly Rental Value'].astype(int)

#Print the column to verify
print(filtered_df['Estimated Monthly Rental Value'])

Respresented below you can see the relationship between costs associated with housing to Rental Values. Houses and departamentos have been isolated for analysis as there is minimal data on some other classes or it is not housing that could be approved for a volunteer to live in. Considering dataset limitations, I kept entries with missing rental values and set them as 0. That way visualization of average housing expenses is possible. Yellow is houses and blue is apartments. By hovering over points you can inspect that entry's data.

# **Interpretation:**

**Baseline Expenses:**  

*   The lowest cluster of points on the y axis, both for houses and apartments, suggests there's a baseline level of total monthly expenses that can be associated with each rental value. This could serve as a guide for the minimum allowance necessary.

**Expense Variability:**

*   The wide distribution along the y axis, indicates that rent alone is not a reliable indicator of total cost of living. This implies that the allowance should account for additional expenses that could significantly vary from one house to another. Considering the volatility of mareket availability. These expenses factored are all basic utilities not discretionary options according to housing checklists and living conditions standards advertised to volunteers in urban environments.

> _The UBIGEO codes' covariance factor would need to be examined in depth to explore these discrepancies.In localities further from Iquitos it seems logical that due to less infrastructure associated utilities cost higher and houses might be a more prevalent option, while higher demand might drive rental prices higher closer to Iquitos. I believe the housing checklist accomodates this with seperate protocol for urban and rural areas._

**Housing Type Consideration:**

* Since houses (yellow points) generally have higher rents and potentially higher total expenses, the budgetary allowance for volunteers might need to be adjusted if a house is being considered for accommodation. However considering the minimal proportion of accomodations that are houses and not apartments. It is reasonable to assume available real estate for a volunteer would be an apartment.

**Upper Limits:**

*   The plot shows that rents above 1500 are associated with houses. If Peace Corps volunteers are not expected to live in such accommodations, the budget can be capped at a level that supports renting lower-cost apartments.

**Budgeting for Apartments:**

*  Most apartments (blue points) fall below a certain rent level and have a relatively more predictable range of total expenses. This could suggest a more standardized allowance for volunteers opting to live in apartments.

**Cost of Living Adjustments:**
The variability in total expenses for a given rent range suggests that setting

*   The variability in total expenses for a given rent range suggests that setting a flat rate for all volunteers might not be equitable. A tiered system or cost of living adjustment could be more appropriate, allowing for higher allowances in cases where total expenses are demonstrably higher.

**Additional Costs:**

* If utilities and other living expenses are not included in the rent (which seems likely given the spread of total expenses), it's important to factor these into the budgetary allowances. In my particular case, the only expense I pay is internet and drinking water.



*Based on these observations, the following recommendations could be made for budgeting:*

1.   Establish a baseline allowance that covers the minimum total monthly expenses associated with the lower-end rent for apartments.
2.   Consider a flexible allowance system that can accommodate the additional expenses of renting a house, should that be a requirement for some volunteers.
3. Implement a system to review economic data in lieu of volunteer surveys where no volunteer surveys are available or no regional coordinators familiar with the local economy or a limited volunteer survey sample is available.
4. Encourage volunteers to opt for apartments due to their lower rent and more predictable expenses, unless a house is specifically needed for their work or personal circumstances.

**By considering these factors, the organization can better tailor its budgetary allowances to the real costs volunteers will face, ensuring they have sufficient support while serving in the city.**

In [ ]:
# @title
import plotly.express as px
plotplot = filtered_df[filtered_df['Housing Type'].isin([1, 2])]

# Create the Scatter Plot
fig = px.scatter(plotplot,
                 x='Estimated Monthly Rental Value',
                 y='Total Expenses',
                 color='Housing Type',  # Color code based on 'Housing Type'
                 hover_data=['Total Monthly Expenses Donated to Household'], # Show this value on hover
                 color_discrete_map={1: 'purple', 2: 'yellow'}) # Setting specific colors

# Update layout if needed
fig.update_layout(title='Rent vs Total Expenses Scatter Plot',
                  xaxis_title='Rent',
                  yaxis_title='Total Monthly Expenses')

# Show plot
fig.show()

In [ ]:
# @title
import numpy as np
unique_rental_values = filtered_df['Estimated Monthly Rental Value'].unique()
print("Unique values in 'Estimated Monthly Rental Value':", unique_rental_values)
sorted_unique_rental_values = np.sort(unique_rental_values)
print("Sorted unique values in 'Estimated Monthly Rental Value':", sorted_unique_rental_values)

Unique values in 'Estimated Monthly Rental Value': [ 600  800  300  400  700   30    0  150  100   60 1000  250 1500  500
  200   50  350   10   80 1200 2000  120   25   20 1300  750  210  280
   90  180  900]
Sorted unique values in 'Estimated Monthly Rental Value': [   0   10   20   25   30   50   60   80   90  100  120  150  180  200
  210  250  280  300  350  400  500  600  700  750  800  900 1000 1200
 1300 1500 2000]


Next I added the Total Expenses to the Monthly Rent, afterwards I filtered for entries less than 900 (the housing portion of our stipend).

In [ ]:
# @title
temp_df = filtered_df[(filtered_df['Estimated Monthly Rental Value'] + filtered_df['Total Expenses']) <= 900]
temp_df.head()

,Tipo de selección del conglomerado,Fecha de Resultado final de la encuesta ( Mes/Día/Año),Exterior Walls: Plastered,Exterior Walls: Painted,Street Type: Dirt Road,Street Type: Asphalted Road,Street Type: Sidewalks,Street Type: Public Lighting,Street Type: None,Housing Type,Main Material of Exterior Walls,Main Material of Floors,Main Material of Roofs,"Total Rooms Excluding Bathroom, Kitchen, Passages, Garage",Rooms Used Exclusively for Sleeping,Construction License,Built with Technical Assistance,Home Ownership Status,Monthly Rent or Purchase Amount,Estimated Monthly Rental Value,Home Has Property Title,Property Title Registered in SUNARP,Credit or Loan for Home Purchase,Main Source of Water,Is Water Drinkable,Chlorine Residual Level,Daily Access to Water Service,Hours of Water Service Per Day,Payment for Water Service,Bathroom Connection Type,Home Lighting Type: Electricity,Electric Service Type,Home Has Cell Phone,Last Monthly Water Consumption Expense,Last Monthly Electricity Consumption Expense,Last Monthly Gas (LPG) Consumption Expense,Last Monthly Telephone Expense,Last Monthly Mobile Phone Expense,Last Monthly Internet Expense,Last Monthly Expense: Other,Monthly Water Expense Paid by Household Member,Monthly Electricity Expense Paid by Household Member,Monthly Gas (LPG) Expense Paid by Household Member,Monthly Mobile Phone Expense Paid by Household Member,Monthly Internet Expense Paid by Household Member,Monthly Other Expense Paid by Household Member,Monthly Portable Internet Expense Paid by Household Member,Monthly Water Expense Gifted or Donated,Monthly Electricity Expense Gifted or Donated,Monthly Gas (LPG) Expense Gifted or Donated,Monthly Mobile Phone Expense Gifted or Donated,Monthly Internet Expense Gifted or Donated,Monthly Other Expense Gifted or Donated,Last Monthly Gas (LPG) Consumption Expense,Total Monthly Expenses Paid by Household,Total Monthly Expenses Donated to Household,Total Monthly Expenses from Self-Supply,Latitude,Longitude,Total Expenses
329,3,20230117,1,3,1.0,0.0,1.0,1.0,0.0,1,9,5,4,4,3,1,2,2,,300,2,NaN,2.0,1,1.0,3,1.0,24,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,68,180,32,46,0,,,0,0,0,0,0.0,,0,378.0,0.0,0,-3.728611,-73.241944,378.0
330,3,20230119,1,1,1.0,0.0,1.0,1.0,0.0,1,1,3,4,2,1,1,1,2,,400,1,1.0,2.0,1,1.0,3,1.0,4,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0,0,68,0,0,,,57,120,0,29,0.0,,0,68.0,278.0,0,-3.728611,-73.241944,346.0
331,3,20230121,1,1,1.0,0.0,1.0,1.0,0.0,1,1,5,4,3,2,2,2,6,,300,,NaN,2.0,1,1.0,3,2.0,,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,63,70,32,55,0,,,0,0,0,0,0.0,,0,250.0,0.0,0,-3.728611,-73.241944,250.0
365,3,20230120,4,3,0.0,1.0,0.0,0.0,0.0,5,7,4,7,4,3,,,2,,30,2,NaN,2.0,8,NaN,3,NaN,,NaN,9,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,,,,30,,0,,,,,0,NaN,11,0,30.0,71.0,0,-4.001667,-73.156944,101.0
368,3,20230119,4,3,0.0,1.0,0.0,0.0,0.0,1,7,4,4,3,2,2,2,2,,30,2,NaN,2.0,8,NaN,3,NaN,,NaN,9,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,,,,,,11,,,,,,NaN,0,0,11.0,0.0,0,-4.001667,-73.156944,11.0


In [ ]:
# @title
temp_df.describe()

,Street Type: Dirt Road,Street Type: Asphalted Road,Street Type: Sidewalks,Street Type: Public Lighting,Street Type: None,Housing Type,Estimated Monthly Rental Value,Property Title Registered in SUNARP,Credit or Loan for Home Purchase,Is Water Drinkable,Daily Access to Water Service,Payment for Water Service,Home Lighting Type: Electricity,Electric Service Type,Home Has Cell Phone,Last Monthly Water Consumption Expense,Last Monthly Electricity Consumption Expense,Last Monthly Gas (LPG) Consumption Expense,Last Monthly Telephone Expense,Last Monthly Mobile Phone Expense,Last Monthly Internet Expense,Last Monthly Expense: Other,Monthly Internet Expense Gifted or Donated,Last Monthly Gas (LPG) Consumption Expense,Total Monthly Expenses Paid by Household,Total Monthly Expenses Donated to Household,Total Expenses
count,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.00000,114.000000,479.0,293.000000,293.000000,363.000000,479.000000,383.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,359.0,479.000000,479.000000,479.000000,479.000000
mean,0.321503,0.392484,0.352818,0.776618,0.098121,1.340292,213.28810,1.070175,2.0,1.098976,1.119454,1.143251,0.799582,1.258486,0.841336,0.816284,0.799582,0.576200,0.006263,0.841336,0.749478,0.415449,0.0,0.018789,146.985386,11.432150,158.417537
std,0.467542,0.488814,0.478346,0.416948,0.297789,1.036502,168.61264,0.256570,0.0,0.299141,0.324877,0.350812,0.400731,0.438375,0.365744,0.387657,0.400731,0.494676,0.078974,0.365744,0.433767,0.493314,0.0,0.236921,119.630418,31.251094,120.971265
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,1.000000,2.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,100.00000,1.000000,2.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.500000,0.000000,0.0,0.000000,42.000000,0.000000,57.500000
50%,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,200.00000,1.000000,2.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.0,0.000000,130.000000,0.000000,146.000000
75%,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,300.00000,1.000000,2.0,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.0,0.000000,214.000000,10.000000,230.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,800.00000,2.000000,2.0,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,3.000000,699.000000,278.000000,768.000000


In [ ]:
import folium

#Initialize a new Folium map
temp_map = folium.Map(location=[-3.75, -73.25], zoom_start=10)

#Iterate through the unique rows in the temp_df
for index, row in temp_df.drop_duplicates(subset=['Latitude', 'Longitude']).iterrows():
    lat = row['Latitude']
    lon = row['Longitude']

    #Calculate the count for each unique map point
    count = temp_df[(temp_df['Latitude'] == lat) & (temp_df['Longitude'] == lon)].shape[0]

    if lat is not None and lon is not None:
        #Create a popup text with the count
        popup_text = f"Count: {count}"

        #Add marker with popup to the map
        folium.Marker([lat, lon], popup=popup_text).add_to(temp_map)

#Display the map
temp_map

In [ ]:
!pip install ipywidgets

### **Charasteristics Based on Rental Value with 900 Budget**

1. Raw
2. Normalized

In [ ]:
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import Layout
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import display
from plotly.subplots import make_subplots

#Creating the DataFrame
data = {
    'Estimated Monthly Rental Value': temp_df['Estimated Monthly Rental Value'],
    'Daily Access to Water Service': temp_df['Daily Access to Water Service'],
    'Home Has Cell Phone': temp_df['Home Has Cell Phone'],
    'Home Has Property Title': temp_df['Home Has Property Title'],
    'Home Lighting Type: Electricity': temp_df['Home Lighting Type: Electricity'],
    'Property Title Registered in SUNARP': temp_df['Property Title Registered in SUNARP'],
    'Built with Technical Assistance': temp_df['Built with Technical Assistance'],
    'Construction License': temp_df['Construction License']
}
df = pd.DataFrame(data)

def interpret_bool(value):
    if value == 1:
        return True
    else:
        return False

#Apply this function to your boolean columns
bool_columns = ['Daily Access to Water Service', 'Home Has Cell Phone', 'Home Has Property Title',
                'Home Lighting Type: Electricity', 'Property Title Registered in SUNARP',
                'Built with Technical Assistance', 'Construction License']

for col in bool_columns:
    df[col] = df[col].apply(interpret_bool)

title_condition = df['Property Title Registered in SUNARP'] | df['Home Has Property Title']

#List of other boolean conditions excluding the two title conditions
other_conditions = ['Daily Access to Water Service', 'Home Has Cell Phone', 'Home Lighting Type: Electricity',
                    'Built with Technical Assistance', 'Construction License']

#'All Conditions Met' is True if all other conditions are True AND either of the title conditions is True
df['All Conditions Met'] = df[other_conditions].all(axis=1) & title_condition

#Group by 'Estimated Monthly Rental Value' and calculate the sum (True counts as 1)
grouped = df.groupby('Estimated Monthly Rental Value').sum()
grouped_proportions = df.groupby('Estimated Monthly Rental Value')[bool_columns + ['All Conditions Met']].mean()

fig1 = px.bar(
    grouped.reset_index(),
    x='Estimated Monthly Rental Value',
    y=bool_columns,
    title='Grouped Bar Chart of Home Characteristics by Rental Value',
    labels={'Estimated Monthly Rental Value': 'Rental Value', 'value': 'Count'},
    height=500
)

plt.figure(figsize=(20, 10))

#Create a bar chart using plotly for the second DataFrame
fig2 = px.bar(
    grouped_proportions.reset_index(),
    x='Estimated Monthly Rental Value',
    y=bool_columns + ['All Conditions Met'],
    title='Proportion of Home Characteristics by Rental Value',
    labels={'Estimated Monthly Rental Value': 'Rental Value', 'value': 'Proportion'},
    height=500
)

fig1.show()
fig2.show()

variable_checkboxes_1 = [widgets.Checkbox(description=col) for col in bool_columns]
variable_checkboxes_2 = [widgets.Checkbox(description=col) for col in bool_columns]

def update_plot_1(**kwargs):
    selected_variables = [checkbox.description for checkbox in variable_checkboxes_1 if checkbox.value]
    fig1.data = []  #Clear existing data on update
    for selected_variable in selected_variables:
        fig1.add_trace(go.Bar(
            x=grouped.reset_index()['Estimated Monthly Rental Value'],
            y=grouped.reset_index()[selected_variable],
            name=selected_variable
        ))

def update_plot_2(**kwargs):
    selected_variables = [checkbox.description for checkbox in variable_checkboxes_2 if checkbox.value]
    fig2.data = []  #Clear existing data on update
    for selected_variable in selected_variables:
        fig2.add_trace(go.Bar(
            x=grouped_proportions.reset_index()['Estimated Monthly Rental Value Normalized'],
            y=grouped_proportions.reset_index()[selected_variable],
            name=selected_variable
        ))

#Initial plot for both subplots
update_plot_1()  #Update the first subplot
update_plot_2()  #Update the second subplot

#Display the checkbox widgets and update the plots when checkboxes change
for checkbox in variable_checkboxes_1:
    checkbox.observe(update_plot_1, 'value')

for checkbox in variable_checkboxes_2:
    checkbox.observe(update_plot_2, 'value')


<Figure size 2000x1000 with 0 Axes>

## **Characteristics Based on Rental Value + Expenses w/in 900 Budget**

1. Raw
2. Normalized

In [ ]:
#Create a temporary DataFrame with modified 'Rental Value'
temp_df['Modified Rental Value'] = temp_df['Estimated Monthly Rental Value'] + temp_df['Total Expenses']

#Creating the DataFrame
data = {
    'Modified Rental Value': temp_df['Modified Rental Value'],
    'Daily Access to Water Service': temp_df['Daily Access to Water Service'],
    'Home Has Cell Phone': temp_df['Home Has Cell Phone'],
    'Home Has Property Title': temp_df['Home Has Property Title'],
    'Home Lighting Type: Electricity': temp_df['Home Lighting Type: Electricity'],
    'Property Title Registered in SUNARP': temp_df['Property Title Registered in SUNARP'],
    'Built with Technical Assistance': temp_df['Built with Technical Assistance'],
    'Construction License': temp_df['Construction License']
}
df = pd.DataFrame(data)

def interpret_bool(value):
    if value == 1:
        return True
    else:
        return False

#Apply this function to boolean columns
bool_columns = ['Daily Access to Water Service', 'Home Has Cell Phone', 'Home Has Property Title',
                'Home Lighting Type: Electricity', 'Property Title Registered in SUNARP',
                'Built with Technical Assistance', 'Construction License']

for col in bool_columns:
    df[col] = df[col].apply(interpret_bool)

title_condition = df['Property Title Registered in SUNARP'] | df['Home Has Property Title']

#List of other boolean conditions excluding the two title conditions
other_conditions = ['Daily Access to Water Service', 'Home Has Cell Phone', 'Home Lighting Type: Electricity',
                    'Built with Technical Assistance', 'Construction License']

# 'All Conditions Met' is True if all other conditions are True AND either of the title conditions is True
df['All Conditions Met'] = df[other_conditions].all(axis=1) & title_condition

grouped = df.groupby('Modified Rental Value').sum()
grouped_proportions = df.groupby('Modified Rental Value')[bool_columns + ['All Conditions Met']].mean()

fig1 = px.bar(
    grouped.reset_index(),
    x='Modified Rental Value',
    y=bool_columns,
    title='Grouped Bar Chart of Home Characteristics by Rental Value',
    labels={'Modified Rental Value': 'Modified Rental Value', 'value': 'Count'},
    height=500
)

plt.figure(figsize=(20, 10))

# Create a bar chart using plotly for the second DataFrame
fig2 = px.bar(
    grouped_proportions.reset_index(),
    x='Modified Rental Value',
    y=bool_columns + ['All Conditions Met'],
    title='Proportion of Home Characteristics by Rental Value',
    labels={'Modified Rental Value': 'Modified Rental Value', 'value': 'Proportion'},
    height=500
)

fig1.show()
fig2.show()

variable_checkboxes_1 = [widgets.Checkbox(description=col) for col in bool_columns]
variable_checkboxes_2 = [widgets.Checkbox(description=col) for col in bool_columns]

def update_plot_1(**kwargs):
    selected_variables = [checkbox.description for checkbox in variable_checkboxes_1 if checkbox.value]
    fig1.data = []  # Clear existing data on update
    for selected_variable in selected_variables:
        fig1.add_trace(go.Bar(
            x=grouped.reset_index()['Estimated Monthly Rental Value'],
            y=grouped.reset_index()[selected_variable],
            name=selected_variable
        ))

def update_plot_2(**kwargs):
    selected_variables = [checkbox.description for checkbox in variable_checkboxes_2 if checkbox.value]
    fig2.data = []  # Clear existing data on update
    for selected_variable in selected_variables:
        fig2.add_trace(go.Bar(
            x=grouped_proportions.reset_index()['Estimated Monthly Rental Value Normalized'],
            y=grouped_proportions.reset_index()[selected_variable],
            name=selected_variable
        ))

# Initial plot for both subplots
update_plot_1()  # Update the first subplot
update_plot_2()  # Update the second subplot

# Display the checkbox widgets and update the plots when checkboxes change
for checkbox in variable_checkboxes_1:
    checkbox.observe(update_plot_1, 'value')

for checkbox in variable_checkboxes_2:
    checkbox.observe(update_plot_2, 'value')

<Figure size 2000x1000 with 0 Axes>

## **Characteristics Based on Rental Value + Expenses of All Samples in Iquitos**

1. Raw
2. Normalized

In [ ]:
# @title
df = filtered_df.copy()

df['Modified Rental Value'] = df['Estimated Monthly Rental Value'] + df['Total Expenses']

def interpret_bool(value):
    if value == 1:
        return True
    else:
        return False

#Apply this function to boolean columns
bool_columns = ['Daily Access to Water Service', 'Home Has Cell Phone', 'Home Has Property Title',
                'Home Lighting Type: Electricity', 'Property Title Registered in SUNARP',
                'Built with Technical Assistance', 'Construction License']

for col in bool_columns:
    df[col] = df[col].apply(interpret_bool)

title_condition = df['Property Title Registered in SUNARP'] | df['Home Has Property Title']

#List of other boolean conditions excluding the two title conditions
other_conditions = ['Daily Access to Water Service', 'Home Has Cell Phone', 'Home Lighting Type: Electricity',
                    'Built with Technical Assistance', 'Construction License']

#'All Conditions Met' is True if all other conditions are True AND either of the title conditions is True
df['All Conditions Met'] = df[other_conditions].all(axis=1) & title_condition

def interpret_bool(value):
    if value == 1:
        return True
    else:
        return False

#Apply this function to your boolean columns
bool_columns = ['Daily Access to Water Service', 'Home Has Cell Phone', 'Home Has Property Title',
                'Home Lighting Type: Electricity', 'Property Title Registered in SUNARP',
                'Built with Technical Assistance', 'Construction License']

for col in bool_columns:
    df[col] = df[col].apply(interpret_bool)

title_condition = df['Property Title Registered in SUNARP'] | df['Home Has Property Title']

# List of other boolean conditions excluding the two title conditions
other_conditions = ['Daily Access to Water Service', 'Home Has Cell Phone', 'Home Lighting Type: Electricity',
                    'Built with Technical Assistance', 'Construction License']

# 'All Conditions Met' is True if all other conditions are True AND either of the title conditions is True
df['All Conditions Met'] = df[other_conditions].all(axis=1) & title_condition

df['Modified Rental Value'] = df['Modified Rental Value'].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

bins = list(range(0, df['Modified Rental Value'].max() + 51, 50))
df['Modified Rental Value Bins'] = pd.cut(df['Modified Rental Value'], bins)

# Group by 'Modified Rental Value' and calculate the sum (True counts as 1)
grouped = df.groupby('Modified Rental Value').sum()
grouped_proportions = df.groupby('Modified Rental Value')[bool_columns + ['All Conditions Met']].mean()

fig1 = px.bar(
    grouped.reset_index(),
    x='Modified Rental Value',
    y=bool_columns,
    title='Grouped Bar Chart of Home Characteristics by Modified Rental Value',
    labels={'Modified Rental Value': 'Modified Rental Value', 'value': 'Count'},
    height=500
)

fig2 = px.bar(
    grouped_proportions.reset_index(),
    x='Modified Rental Value',
    y=bool_columns + ['All Conditions Met'],
    title='Proportion of Home Characteristics by Modified Rental Value',
    labels={'Modified Rental Value': 'Modified Rental Value', 'value': 'Proportion'},
    height=500
)

# Enable zooming with the "Zoom Mode" button in the toolbar
fig1.update_xaxes(fixedrange=False)
fig2.update_xaxes(fixedrange=False)

# Show the figures
fig1.show()
fig2.show()

## **Housing Material to Costs Matrix**

In [ ]:
df = filtered_df.copy()

roof_material_labels = {
    '1': 'Concreto',
    '2': 'Madera',
    '3': 'Tejas',
    '4': 'Planchas de calamina, fibra de cemento o similares',
    '5': 'Caña o estera con torta de barro o cemento',
    '6': 'Triplay/estera/carrizo',
    '7': 'Paja, hojas de palmera',
    '8': 'Otro material'
}

wall_material_labels = {
    '1': 'Ladrillo o bloque de cemento',
    '2': 'Piedra o sillar con cal o cemento',
    '3': 'Adobe',
    '4': 'Tapia',
    '5': 'Quincha (caña con barro)',
    '6': 'Piedra con barro',
    '7': 'Madera (pona, tornillo, etc)',
    '8': 'Triplay/calamina/estera',
    '9': 'Otro material'
}

df['Modified Rental Value'] = df['Estimated Monthly Rental Value'] + df['Total Expenses']
df['Modified Rental Value'] = df['Modified Rental Value'].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

df['Main Material of Roofs'] = df['Main Material of Roofs'].map(roof_material_labels)
df['Main Material of Exterior Walls'] = df['Main Material of Exterior Walls'].map(wall_material_labels)

df['Roof Material Code'] = df['Main Material of Roofs'].astype('category').cat.codes
df['Wall Material Code'] = df['Main Material of Exterior Walls'].astype('category').cat.codes
#Create the 3D scatter plot

fig = go.Figure(data=[go.Scatter3d(
    x=df['Modified Rental Value'],
    y=df['Roof Material Code'],
    z=df['Wall Material Code'],
    mode='markers',
    marker=dict(
        size=3,
        color=df['Modified Rental Value'],  #Set color equal to 'Modified Rental Value'
        colorscale='Jet',  #Heat color scale
        colorbar=dict(title='Rental Value')
    )
)])

#Customize layout
fig.update_layout(
    title='Rental Values by Roof and Wall Materials',
    scene=dict(
        xaxis_title='Modified Rental Value',
        yaxis_title='Roof Material Code',
        zaxis_title='Wall Material Code',
        xaxis=dict(title_font=dict(size=10), tickfont=dict(size=8)),
        yaxis=dict(tickvals=list(roof_material_labels.keys()), ticktext=list(roof_material_labels.keys()), tickfont=dict(size=8)),
        zaxis=dict(tickvals=list(wall_material_labels.keys()), ticktext=list(wall_material_labels.keys()), tickfont=dict(size=8))
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

#Add custom 2D legend for categories on the left side of the plot
annotations = []
for i, (key, val) in enumerate(roof_material_labels.items()):
    annotations.append(dict(
        x=0.05, y=1.0-(i*0.05), xref='paper', yref='paper',
        text=f"Roof {key}: {val}", showarrow=False
    ))

for i, (key, val) in enumerate(wall_material_labels.items()):
    annotations.append(dict(
        x=0.05, y=0.5-(i*0.05), xref='paper', yref='paper',
        text=f"Wall {key}: {val}", showarrow=False
    ))

fig.update_layout(annotations=annotations)

fig.show()